In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo")

In [3]:
from govrfp.rfpinsights.llmannotations import llmannotations

test_query = "What is the purpose of the RFP?"
llmannotations(test_query, llm)

{'is_formatting_compliance': False,
 'is_finance_compliance': False,
 'is_legal_compliance': False,
 'is_technical_compliance': False,
 'is_mandatory': False,
 'other': True}

In [4]:
test_data_path = "tests/mar-21/03-all-compliances.txt"
with open(test_data_path, 'r') as file:
    test_data = file.readlines()
test_data = [line.strip() for line in test_data]
test_data = [line for line in test_data if line]
test_data[:5]

['- Provide the mailing address of the authorized government representative',
 '- Include the date and signature of the authorized government representative',
 '- Include the S/R voucher number, S/R account number, and check number for payment',
 '- Provide the e-mail of the authorized government representative',
 '- Include the total number of containers received, date received, location received at, and name of the person who received it']

In [5]:
import tqdm
annotations = []
for line in tqdm.tqdm(test_data):
    annotations.append(llmannotations(line, llm))

  0%|          | 0/90 [00:00<?, ?it/s]

100%|██████████| 90/90 [00:51<00:00,  1.74it/s]


In [6]:
format = []
finance = []
legal = []
technical = []
other = []

for index, annotation in enumerate(annotations):
    if annotation['is_formatting_compliance']:
        format.append(test_data[index])
    if annotation['is_finance_compliance']:
        finance.append(test_data[index])
    if annotation['is_legal_compliance']:
        legal.append(test_data[index])
    if annotation['is_technical_compliance']:
        technical.append(test_data[index])
    if annotation['is_mandatory'] or annotation['other']:
        other.append(test_data[index])

len(format), len(finance), len(legal), len(technical), len(other)

(17, 3, 29, 12, 50)

In [7]:
import pandas as pd

format_df = pd.DataFrame(format, columns=["format"])
finance_df = pd.DataFrame(finance, columns=["finance"])
legal_df = pd.DataFrame(legal, columns=["legal"])
technical_df = pd.DataFrame(technical, columns=["technical"])
other_df = pd.DataFrame(other, columns=["other"])

In [8]:
# export xlsx file with different sheets
with pd.ExcelWriter('tests/mar-21/04-compliances.xlsx') as writer:
    format_df.to_excel(writer, sheet_name='COMPLIANCES-format', index=True)
    finance_df.to_excel(writer, sheet_name='COMPLIANCES-finance', index=True)
    legal_df.to_excel(writer, sheet_name='COMPLIANCES-legal', index=True)
    technical_df.to_excel(writer, sheet_name='COMPLIANCES-technical', index=True)
    other_df.to_excel(writer, sheet_name='COMPLIANCES-other', index=True)